# Fine-tuning YOLOv8 with External Dataset and Exporting to TFLite
## Import required packages

In [ ]:
from ultralytics import YOLO
import os 
import matplotlib.pyplot as plt
import zipfile

## Load the Pre-Trained YOLOv8 Model

In [ ]:
model = YOLO('yolov8n.pt')

## Fine-Tune the Model 

In [ ]:
# Define the folder paths
FOLDER_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATAFOLDER = os.path.join(FOLDER_PATH, 'sentinel_model_gen', 'data-images')

# unzip data files into the specified folder
def unzip_data(zip_file, folder_path):
    # created folder if it does not exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # unzip the contents of the zip file to the destination folder
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(folder_path)

    print(f"{zip_file} unzip to {folder_path}")

# fine tune the YOLO model with new dataset
def fine_tune(model, yaml_path):
    model.train(data=yaml_path, epochs=50, imgsz=640, batch=16, device=0)
    return model

# save the model 
def save_model(model):
    model.save('yolo_fine_tuned.pt')

# unzip parcel images dataset into the 'parcel-images' folder
unzip_data(os.path.join(DATAFOLDER, "parcel.v1i.yolov8.zip"), os.path.join(DATAFOLDER, "parcel-images"))

# unzip the pistol images dataset into the 'pistol-images'folder
unzip_data(os.path.join(DATAFOLDER, "Pistols.v1-resize-416x416.yolov8.zip"), os.path.join(DATAFOLDER, "pistol-images"))

# path to yaml configuration for parcel images
parcel_yaml = os.path.join(DATAFOLDER, "parcel-images", "data.yaml")

# path to yaml configuration for pistol images
pistol_yaml = os.path.join(DATAFOLDER, "pistol-images", "data.yaml")

# fine tune the YOLO model on parcel images 
parcel_ft_model = fine_tune(model, parcel_yaml)

# further fine tune the previously fine funed model
pistol_ft_model = fine_tune(parcel_ft_model, pistol_yaml)

# save the model
save_model(pistol_ft_model)

## Load the Fine-Tuned Model

In [ ]:
def load_model(model_name):
    model = YOLO(model_name)
    return model

fined_tuned_model = load_model('yolo_fine_tuned.pt')

## Test Model 

In [ ]:
def test_model(model, img_path):
    # Perform object detection
    results = model(img_path)

    # retrieve the annotated image (with bounding boxes and labels)
    annotated_img = results[0].plot()

    # display the image
    plt.imshow(annotated_img)
    plt.axis('off')
    plt.show()


test_model(fined_tuned_model, os.path.join(FOLDER_PATH, 'sentinel_model_gen', 'guy_w_box.png'))
test_model(fined_tuned_model, os.path.join(FOLDER_PATH, 'sentinel_model_gen', 'guy.png'))

## Export the model to TFLite Format

In [ ]:
# export the model to TFLite for use in the detection system 
def export_model(model): 
    model.export(format='tflite')